# 评价分类结果（上）

## 1.2 精准率和召回率

### 精准率为我们关注的那个事件，预测的有多准（猜的情况中猜中的概率）

### 召回率为我们关注的那个事件真实的发生情况下，我们成功预测的比例是多少（猜中的个数占总体该个数的比率）

#### 代码实现：混淆矩阵

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

In [7]:
# 要构造偏斜数据，将数字9 的对应索引的元素设置为1，0-8设置为0
y

array([0, 1, 2, ..., 8, 9, 8])

In [9]:
y[digits.target == 9] = 1

In [10]:
y[digits.target == 9]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [12]:
y[digits.target != 9] = 0

In [13]:
y

array([0, 0, 0, ..., 0, 1, 0])

In [14]:
# 使用逻辑回归做一个分类
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [15]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
# 得到X_test所对应的预测值
y_log_predict = log_reg.predict(X_test)

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [26]:
log_reg.score(X_test, y_test)

0.9755555555555555

#### 定义混淆矩阵的四个指标：TN

In [27]:
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    # (y_true == 0)向量与数值按位比较，得到的是一个布尔向量
    # 向量与向量按位与，结果还是布尔向量
    # np.sum计算布尔向量中True的个数（True记为1， False记为0）
    return np.sum((y_true == 0) & (y_predict == 0)) # 向量与向量按位与，结果还是向量
TN(y_test, y_log_predict)

403

#### 定义混淆矩阵的四个指标：FP

In [28]:
def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    # (y_true == 0):向量与数值按位比较，得到的是一个布尔向量
    # 向量与向量按位与，结果还是布尔向量
    # np.sum 计算布尔向量中True的个数（True记为1，False记为0）
    return np.sum((y_true == 0) & (y_predict == 1)) # 向量与向量按位与，结果还是向量
FP(y_test, y_log_predict)

2

#### 定义混淆矩阵的四个指标：FN

In [29]:
def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    # (y_true == 0):向量与数值按位比较，得到的是一个布尔向量
    # 向量与向量按位与，结果还是布尔向量
    # np.sum 计算布尔向量中True的个数（True记为1， False记为0）
    return np.sum((y_true == 1) & (y_predict == 0)) # 向量与向量按位与，结果还是向量）
FN(y_test, y_log_predict)

9

#### 定义混淆矩阵的四个指标：TP

In [33]:
def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    # (y_true == 0):向量与数值按位比较，得到的是一个布尔向量
    # 向量与向量按位与，结果还是布尔向量
    # np.sum 计算布尔向量中True的个数（True记为1， False记为0）
    return np.sum((y_true == 1) & (y_predict == 1)) # 向量与向量按位与，结果还是向量
TP(y_test, y_log_predict)

36

In [34]:
def confusion_matrix(y_true, y_predict):
    return np.array([
        [TN(y_true, y_predict), FP(y_true, y_predict)],
        [FN(y_true, y_predict), TP(y_true, y_predict)]
    ])
confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

#### 精准率

In [36]:
def precision_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fp = FP(y_true, y_predict)
    try:
        return tp/(tp+fp)
    except:
        return 0.0
precision_score(y_test, y_log_predict)

0.9473684210526315

#### 召回率 

In [38]:
def recall_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp/(tp+fn)
    except:
        return 0.0
recall_score(y_test, y_log_predict)

0.8

### scikit-learn中的混淆矩阵、精准率和召回率

In [40]:
# 混淆矩阵
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]], dtype=int64)

In [44]:
# 精准率
from sklearn.metrics import precision_score
precision_score(y_test, y_log_predict)

0.9473684210526315

In [45]:
# 召回率
from sklearn.metrics import recall_score
recall_score(y_test, y_log_predict)

0.8